📁 Resources

In [1]:
import pandas as pd
import time
import random
import dataextension as dx
import requests
# from pandasgui import show
from config.settings import DATA_DIR
from config.settings import URLS_DIR
import emailaddress


keywords = pd.read_csv(rf"{DATA_DIR}keywords.csv")["selection"].dropna().tolist()
bannedwords = pd.read_csv(rf"{DATA_DIR}bannedwords.csv")["job_title_de"].dropna().to_list()

c:\Users\besff\AppData\Local\pypoetry\Cache\virtualenvs\centback-nvo6Z_mp-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📁 Load links

In [2]:

links  = pd.read_csv(rf"{URLS_DIR}europade_urls.csv")
df = links[["url","data","headers"]][:10000]
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Generate initial content

In [3]:

def icontent(row):
    time.sleep(random.randint(1,3))
    response = requests.post(row["url"], json=eval(row["data"]), headers=eval(row["headers"]))
    return str(response.json()['jvs'])
 
    
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.insertone(with_func=icontent,column_name='icontent', description="Initial Content")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)
df.to_csv(rf"{DATA_DIR}europade_backup.csv",index=False)

Initial Content: 100%|██████████| 10000/10000 [8:03:36<00:00,  2.90s/it]  


📁 Convert icontent to list

In [4]:

def convert_to_list(x):
    return eval(x)
 
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df["icontent"] = df["icontent"].progress_apply(convert_to_list)
df = df[df["icontent"].apply(lambda x: not dx.is_blank(x))]
df = df.explode("icontent",ignore_index=True)
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)
df.info()
 

Initial Content: 100%|██████████| 10000/10000 [00:00<00:00, 187074.51it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   url       0 non-null      object
 1   data      0 non-null      object
 2   headers   0 non-null      object
 3   icontent  0 non-null      object
dtypes: object(4)
memory usage: 132.0+ bytes


📁 Extract Job Title de

In [5]:

def job_title_de(row): 
    return eval(row["icontent"])["title"]

df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.insertone(with_func=job_title_de,column_name='job_title_de', description="job_title_de")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

job_title_de: 1it [00:00, ?it/s]


📁 Select positive Job Title de

In [6]:
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.select_positive(words=keywords)
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

KeyError: 'job_title_de'

📁 Delete Negative Job Title de

In [ ]:
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.delete_negative(words=bannedwords)
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get Company Name "company_name"

In [ ]:
def company_name(row):
    return eval(row['icontent'])['employer']['name']

df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.insertone(with_func=company_name,column_name='company_name', description="company_name")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Drop duplicates base on "company_name" and "job_title"

In [ ]:
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.drop_duplicates(subset=['company_name','job_title_de'],keep='first')
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Extract Job Title EN "job_title_en"

In [ ]:
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df['job_title_en'] = df['job_title_de']
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get Source url "source_url"

In [ ]:
def source_url(row):
    jobid =  eval(row['icontent'])['id']
    return f"https://ec.europa.eu/eures/portal/jv-se/jv-details/{jobid}?jvDisplayLanguage=de&lang=en"
   
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.insertone(with_func=source_url,column_name='source_url', description="source_url")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get json Source url "jsource_url"

In [ ]:
def jsource_url(row):
    jobid =  eval(row['icontent'])['id']
    return f"https://ec.europa.eu/eures/eures-apps/searchengine/page/jv/id/{jobid}?lang=en"
   
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.insertone(with_func=jsource_url,column_name='jsource_url', description="jsource_url")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get Job Description DE "job_description_de"

In [ ]:
def job_description_de(row): 
    return eval(row["icontent"])["description"]
    
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.insertone(with_func=job_description_de,column_name='job_description_de', description="job_description_de")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get Job Description EN "job_description_en"

In [ ]:
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df['job_description_en'] = df['job_description_de']
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get employment_type "job_employment_type"

In [ ]:
def employment_type(row): 
    return eval(row['icontent'])['positionScheduleCodes'][0]
     
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.insertone(with_func=employment_type,column_name='employment_type', description="employment_type")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get Company Website "company_website"

In [ ]:
def company_website(row):
    return eval(row['icontent'])['employer']['website']
  
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.insertone(with_func=company_website,column_name='company_website', description="company_website")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get Get second page json content "jcontent"

In [ ]:
def jcontent(row):
    time.sleep(random.randint(1,3))
    response = requests.get(row["jsource_url"])
    return str(response.json())
 
    
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.insertone(with_func=jcontent,column_name='jcontent', description="jcontent")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)
df.to_csv(rf"{DATA_DIR}europade_backup.csv",index=False)

📁 Get City "city"

In [ ]:
def city(row):
    return eval(row['jcontent'])['jvProfiles']['de']['locations'][0]['cityName']
  
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.insertone(with_func=city,column_name='city', description="city")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get Full Name "full_name"

In [ ]:
def full_name(row):
    first_name = eval(row['jcontent'])['jvProfiles']['de']['personContacts'][0].get('givenName','')
    last_name = eval(row['jcontent'])['jvProfiles']['de']['personContacts'][0].get('familyName','')
    return f"{first_name} {last_name}"
  
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.insertone(with_func=full_name,column_name='full_name', description="full_name")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get contact_email "contact_email"

In [ ]:
def contact_email(row):
    email = emailaddress.from_text(text=row['job_description_de'])
    if email:
        return email[0]
  
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.insertone(with_func=contact_email,column_name='contact_email', description="contact_email")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)
df.info()